### Lee filter 

##### Lee filter as defined in Lee, J.S., 1980, Digital Image Enhancement and Noise Filtering by Use of Local Statistics, IEEE Transactions on Pattern Analysis and Machine Intelligence, Vol. PAMI-2, No. 2. The Lee filter makes use of local mean and variance statistics for each pixel for estimating the minimum mean-square error (MMSE) and apply it for obtaining an estimate of the noise. This filter uses an adaptive method, with a sliding kernel used for determining local statistcs.  While in the forementioned paper three cases of noise reduction are treated (additive, multiplicative and combination), the following implementation is suitable only for multiplicative noise cases that introduces an effect to images, similar to the "salt and pepper" effect of speckle in Synthetic Apperture Radar (SAR) imagery. (Note that speckle in SAR is not noise, but interference of incoming signal from different scatterers). 

In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import math
from skimage import exposure
import os
import glob

In [ ]:
def lee_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    [cols, rows]=img_array.shape

    # equalizing the histogram
    '''''val1,val2 = np.percentile(img_array, (2,90))
    new_img= exposure.rescale_intensity(img_array, in_range=(val1,val2))
    plt.figure(figsize=(7, 7))                   
    fig = plt.imshow(arr_new, cmap = 'gray')  
    plt.axis(False)'''''
    
    eq=exposure.equalize_hist(img_array).astype(np.float32)
    
    #setup requirements for Lee Filter
    
    window_size=5   #for window size use an odd number
    number_looks = 4.4 #default for Sentinel-1 (2018)
    mean_variance=1/number_looks
    mean_noise= 1  #default value for multiplicative noise is 1
    
    mean=cv.blur(img_array, (window_size, window_size))                                               
    mean_sq = cv.blur(img_array*img_array, (window_size, window_size))
    local_variance= mean_sq-mean*mean
    
    output_array = np.zeros(img_array.shape, dtype=np.float32)
    
    # define filter function
    
    def lee():
        for i in range(cols):
            for j in range(rows):
                center_pixel=img_array[i,j]
                weight= (mean_noise*local_variance[i,j])/((mean[i,j]*mean[i,j]*mean_variance)+(mean_noise*mean_noise*local_variance[i,j]))
                output_array[i,j]=mean[i,j]+weight*(center_pixel-mean_noise*mean[i,j])
        return output_array

    #filter image
    
    filtered_image= lee()
    
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)

In [ ]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/NRCS/*tif'):
    output_file = input_file.split(os.sep)
    output_file[-1] = output_file[-1][:-4] + '_Lee_5.tif'
    output_file = output_file[:-2] + ['Lee_Python'] + output_file[-2:]
    output_file = os.sep.join(output_file)
    lee_filter(input_file, output_file)